# TensorFlow Playground
Welcome to the TensorFlow Playground! This notebook is modeled after the [TensorFlow Playground we application](http://playground.tensorflow.org) which offers a playful way to test different feed forward neural network configurations. Although the app was created by the TensorFlow guys, it does not actually use the library itself. This notebook gives you the opportunity to actually implement different network configurations you might already have tested in the web app. Along this notebook you will also find implementations of the playground in Keras and Theano. Tensorflow lies somewhere inbetween the two in a sense of abstractness.

## Data
As with the original web application you have the choice between four different datasets. The next cell visualizes them and in the cell right below you can specify your choice. To use anything but the gauss dataset you can simply exchange the `datasets.gauss` function call with either `datasets.circle`, `datasets.xor` or `datasets.spiral`. Additionally you can add some noise to the data to make the problem more difficult. Some information about the provided data structures:

The two `*_data` variables contain the actual data in 2 dimensional numpy arrays or shape `(*,2)`. The first axis describes individual observations and the second axis the `x` and `y` values. The `*_labels` variables are simply `(*,1)` numpy arrays specifying a label, `0` or `1`, for each sample. The size of the first axis for both results from the train/test split.

As the names suggest, `train_data` and `train_labels` are designated for training and contain a lot more data (by default $9/10$) and `test_data` and `test_labels` are meant for evaluating the networks performance. You can change the amount of samples inside the `gauss` (or whatever you chose) function call and also adjust the split between training and testing data by adjusting the second parameter for the `datasets.split` function.

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
from playground import DataGenerator

datasets = DataGenerator()

fig = plt.figure('Available Datasets')
for i, name in enumerate(['circle', 'xor', 'gauss', 'spiral']):
    data, labels = getattr(datasets, name)(200)
    axis = plt.subplot(221+i)
    axis.set_title(name)
    axis.scatter(*zip(*data), c=labels, cmap='bwr')
fig.canvas.draw()

In [ ]:
# datasets.circle(), datasets.xor(), datasets.gauss(), datasets.spiral()
data, labels = datasets.gauss(n=1000, noise=0)
(train_data, train_labels), (test_data, test_labels) = datasets.split((data, labels), 10)

fig = plt.figure('Chosen dataset')
plt.scatter(*zip(*data), c=labels, cmap='bwr')
fig.canvas.draw()

## It's your turn!
In the next cell stuff gets interesting. We preimplemented a basic model which is sufficiently powerful for the gauss dataset. It consists of 2 neurons in the input layer to feed `x` and `y` values into, 4 neurons in the single hidden layer and a single neuron in the output layer.

Instead of using buttons and dropdowns you will need to get your hands dirty here in order to cope with more complex datasets! While we provide some guidance to mock the experience of the web application it will be also super useful to checkout the [TensorFlow API documentation](https://www.tensorflow.org/versions/r0.9/api_docs/index.html). Because the documentation does not give any theoretical background on all those different functions you might also want to regularily consultate Wikipedia.

Some ideas on what you can play around with:

  * First take a general look through the code and play with stuff which is not TensorFlow specific. You can for example change the amount of training epochs and the size of the batch the network is trained with in every iteration. Also: If your computer struggles with the computations you might want to consider removing the progress visualization.
  * Adjust the hidden layer, play around with more or even fewer neurons.
  * Add another hidden layer or remove the existing one. You will need to adjust the matrix multiplications for its surrounding layers in order to incorporate the change into the network.
  * Change the activation functions. By default both, the hidden and the output layer, use the sigmoid activation function. Similar to the web application you can use `relu` or `tanh`. You might also consider trying to implement your own `linear` activation function -- TensorFlow does not provide one by itself.
  * Take a look into different error measures and optimizers. This is nothing you can change in the playground web application but actually a powerful tool to tweak your networks performance through better training. Checkout the [optimizers chapter](https://www.tensorflow.org/versions/r0.9/api_docs/python/train.html#optimizers) in the documentation.

When you solved all the datasets you can also start adding noise to the data or take less data for training by adjusting the test/train split (see the code cell above).

In [ ]:
import tensorflow as tf
from numpy.random import permutation
from playground import viz

x = tf.placeholder(tf.float32, shape=[None, 2], name='input_layer')

W_h = tf.Variable(tf.random_normal([2,4]), name='hidden_weights')
b_h = tf.Variable(tf.zeros([4]), name='hidden_bias')
h = tf.nn.sigmoid(tf.matmul(x,W_h) + b_h, name='hidden_layer')

W_y = tf.Variable(tf.random_normal([4,1]), name='output_weights')
b_y = tf.Variable(tf.zeros([1]), name='output_bias')
y = tf.nn.sigmoid(tf.matmul(h,W_y) + b_y, name='output_layer')

t = tf.placeholder(tf.float32, shape=[None, 1], name='targets')

error = tf.reduce_mean(tf.squared_difference(t,y), name='mean_squared_error')
optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(error, name='gradient_descent')

prediction_check = tf.equal(tf.round(y), t, name='prediction_check')
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32), name='accuracy_measure')

sess = tf.Session()
sess.run(tf.initialize_all_variables())

fig = plt.figure('Result')
epochs = 1000
for epoch in range(epochs):
    split = permutation(len(train_labels))
    batch_labels = train_labels[split[:10]]
    batch_data = train_data[split[:10]]
    sess.run(optimizer, {x: batch_data, t: batch_labels})
    if (epoch+1)%100 == 0 or (epoch+1) == epochs:
        perf = sess.run(accuracy, {x: test_data, t: test_labels})
        predictions = sess.run(y, {x: data})
        viz(data, predictions, (epoch+1)/epochs, perf, fig, discretize=False)